<a href="https://colab.research.google.com/github/siddadel/kalidas/blob/main/Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Dec  4 08:01:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers --quiet
!git clone https://github.com/allenai/comet-atomic-2020
!pip install -r ./comet-atomic-2020/requirements.txt --quiet
!wget https://storage.googleapis.com/ai2-mosaic-public/projects/mosaic-kgs/comet-atomic_2020_BART.zip
!unzip comet-atomic_2020_BART.zip

     |████████████████████████████████| 3.1 MB 7.8 MB/s 
     |████████████████████████████████| 895 kB 62.8 MB/s 
     |████████████████████████████████| 3.3 MB 59.5 MB/s 
     |████████████████████████████████| 61 kB 638 kB/s 
     |████████████████████████████████| 596 kB 69.4 MB/s 
Cloning into 'comet-atomic-2020'...
remote: Enumerating objects: 184, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 184 (delta 56), reused 90 (delta 32), pack-reused 56
Receiving objects: 100% (184/184), 7.16 MiB | 32.28 MiB/s, done.
Resolving deltas: 100% (62/62), done.
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 90 kB 7.2 MB/s 
     |████████████████████████████████| 313 kB 40.7 MB/s 
     |████████████████████████████████| 9.1 MB 64.9 MB/s 
     |████████████████████████████████| 379 kB 85.3 MB/s 
     |████████████████████████████████| 829 kB 75.0 MB/s 
     |████████████████████

In [48]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')
data_root = "/content/drive/MyDrive/ANLP21/"
output_dir = "/content/drive/MyDrive/ANLP21/train_concats/"
output_file = data_root+'concatenated.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
import sys
sys.path.insert(1,r'./comet-atomic-2020/models/comet_atomic2020_bart')
from generation_example import Comet
print("model loading ...")
comet = Comet("comet-atomic_2020_BART")
comet.model.zero_grad()
print("model loaded")

model loading ...
model loaded


In [50]:
import pandas as pd
cf = pd.read_csv(data_root+'character_tour_manual_part1_split.csv')
cf = cf[cf.apply(lambda row: row['dev_train_test'] in ['test' , 'train' ,'dev'], axis=1)]
cf['film'] = cf['film'].apply(lambda film: film.replace(".txt.txt.txt.txt.txt.txt.txt",".txt"))
cf_list = cf['film'].to_list()

In [51]:
def film_name_condition(film_name):
  return film_name in cf_list


In [52]:
import pandas as pd
df = pd.read_csv(data_root+"database.csv")
df = df[df.apply(lambda row: film_name_condition(row['film']), axis=1 )]

In [53]:
df

,Unnamed: 0,film,scene,item_ids,item,character,protagonist
0,0,trainspotting.txt,1,1,TRAINSPOTTING,NaN,RENTON
1,1,trainspotting.txt,1,2,Screenplay by John Hodge,NaN,RENTON
2,2,trainspotting.txt,1,3,Based on the Novel by Irvine Welsh,NaN,RENTON
3,3,trainspotting.txt,1,4,Shooting Draft,NaN,RENTON
4,4,trainspotting.txt,2,1,STREET - DAY,NaN,RENTON
...,...,...,...,...,...,...,...
895278,895278,rko_281.txt,76,9,"WELLES says, ""Will burn. Burn up. Burn out. B...",WELLES,WELLES
895279,895279,rko_281.txt,76,10,He looks at Mank and toasts.,NaN,WELLES
895280,895280,rko_281.txt,76,11,"WELLES says, ""Cheers.""",WELLES,WELLES
895281,895281,rko_281.txt,76,12,And Orson Welles smiles. Indomitable.,NaN,WELLES


In [54]:
relations = [ "xAttr",  "MotivatedByGoal", "xReact"]

In [47]:
def infer(item):
  queries= ["{} {} [GEN]".format(item, relation) for relation in relations]
  results = comet.generate(queries, decode_method="beam", num_generate=5)
  return results

film_group = df.groupby('film')
df_concat = []
film_names = []
for film, group in film_group:
    print(film)
    #The following three ways were quicker ways but they don't work for some reason
    # if you want to work with these make sure to convert the results array in infer into json or string before returning
    # group['inference'] = comet.generate(["{} {} [GEN]".format(group['item'], relation) for relation in relations], decode_method="beam", num_generate=5)
    # group['inference'] = group['item'].apply((lambda x: infer(x)), axis=1)
    # group['inference'] = infer(group['item'])
    # for i in range(len(relations)):
    #   # group[str(relations[i])] = group['inference'].apply((lambda x: parse_inf(x, i)), axis=1)
    #   group[str(relations[i])] = parse_inf(group['inference'], i)
    for i in range(len(relations)):
      group[relations[i]] = ["" for i in range(len(group))]

    rel_inf = []
    for ind in group.index:
      item = group['item'][ind]
      results = infer(item)
      for i in range(len(relations)):
        group[relations[i]][ind] = results[i]
    
    df_concat.append(group)
    film_names.append(film)
    group.to_csv(output_dir+film+'.csv')
    # files.download(output_dir+film+'.csv')

10_things_i_hate_about_you.txt


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2001_a_space_odyssey.txt
a_nightmare_on_elm_street.txt
airplane.txt


KeyboardInterrupt: ignored

In [ ]:
import pandas as pd 

df = pd.concat(df_concat)
df.to_csv(output_file)
files.download(output_file)
df